You might need to install this on your system:

apt-get install python3-opencv git

In [ ]:
import os

if not os.path.isdir('k'):
 !git clone https://github.com/joaopauloschuler/k-neural-api.git k
else:
 !cd k && git pull

!cd k && pip install .

In [ ]:
import cai.layers
import cai.datasets
import cai.models
import cai.densenet
import cai.efficientnet
import numpy as np
from tensorflow import keras
from tensorflow.keras import backend
from tensorflow.keras import layers
import gc
import multiprocessing
import random
import tensorflow as tf
print("Tensorflow version:", tf.version.VERSION)
print("Keras version:", keras.__version__)
import PIL
import cv2
#print("Pillow version:", PIL.PILLOW_VERSION)
from sklearn.metrics import classification_report

Tensorflow version: 2.4.1
Keras version: 2.4.0


In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5


In [ ]:
if not os.path.isdir('up'):
  !git clone https://github.com/joaopauloschuler/PlantDoc-Object-Detection-Dataset.git up

Cloning into 'up'...
remote: Enumerating objects: 5190, done.
remote: Total 5190 (delta 0), reused 0 (delta 0), pack-reused 5190
Receiving objects: 100% (5190/5190), 941.89 MiB | 80.51 MiB/s, done.
Resolving deltas: 100% (2328/2328), done.
Checking out files: 100% (5199/5199), done.


In [ ]:
up_data_dir = os.getcwd()+"/up/TRAIN"
up_test_dir = os.getcwd()+"/up/TEST"
data_dir = "cropped_train/"
test_dir = "cropped_test/"
all_dir = "cropped_all/"

In [ ]:
def CropImages(orig_data_dir, dest_data_dir, csv_file, include_original=False, verbose=True, prefix='img_'):
  # Load the CSV file.
  with open (csv_file, "r") as myfile:
    csv_lines=myfile.readlines()

  # Transform the CSV file into an array  
  line_count = 0
  a_lines = []
  for str_line in csv_lines:
    if line_count > 0: 
      a_line = str_line.replace('\n', '').split(',')
      a_lines.append(a_line)
    line_count = line_count + 1
  
  # create destination folder
  if not os.path.isdir(dest_data_dir):
    os.mkdir(dest_data_dir)

  # Save cropped images
  line_count = 0
  failed_count = 0
  original_count = 0
  last_image_file = ''
  for a_line in a_lines:
    dest_folder_name = dest_data_dir+'/'+a_line[3]
    dest_file_name = dest_folder_name+'/'+prefix+str(line_count)+'.jpg'
    orig_file_name = orig_data_dir+'/'+ a_line[0]
    print(line_count,':', a_line, dest_file_name)
    if last_image_file != orig_file_name:
      if os.path.isfile(orig_file_name):
        img = cv2.imread(orig_file_name)
        #cv2_imshow(img)
        last_image_file = orig_file_name
        can_load = True
        original_count = original_count + 1
        if verbose: print("Original imgage shape",img.shape)
        if (include_original):
          if not os.path.isdir(dest_folder_name):
            os.mkdir(dest_folder_name)
          cv2.imwrite(dest_folder_name+'/ori_'+prefix+str(original_count)+'.jpg',img)  
      else:
        if verbose: print('File does not exist:', orig_file_name)
        can_load = False
        #break
    if (can_load):
      y1 = int(a_line[4])
      x1 = int(a_line[5])
      y2 = int(a_line[6])
      x2 = int(a_line[7])

      x_min = np.min([x1, x2])
      x_max = np.max([x1, x2])
      y_min = np.min([y1, y2])
      y_max = np.max([y1, y2])

      crop_img = img[x_min:x_max, y_min:y_max]
      if verbose: print("Cropped imgage shape",crop_img.shape,':', x_min, x_max, y_min, y_max,' deltas:',x_max - x_min, y_max - y_min)
      if not os.path.isdir(dest_folder_name):
        os.mkdir(dest_folder_name)
      if ( (crop_img.shape[0]==0) or (crop_img.shape[1]==0)):
        failed_count = failed_count + 1
        if verbose: print("Failed cropping.");
      else:
        cv2.imwrite(dest_file_name,crop_img)
    else:
      failed_count = failed_count + 1
    line_count = line_count + 1
  print("Processed files:", line_count, "Failed count: ", failed_count, "Original count:", original_count)

In [ ]:
if not os.path.isdir(data_dir):
  CropImages(orig_data_dir=up_data_dir, dest_data_dir=data_dir, csv_file='up/train_labels.csv', include_original=False, prefix='train_')

0 : ['cherry-tree-leaves-and-fruits.jpg', '350', '300', 'Cherry leaf', '198', '77', '299', '252'] cropped_train//Cherry leaf/train_0.jpg
Original imgage shape (300, 350, 3)
Cropped imgage shape (175, 101, 3) : 77 252 198 299  deltas: 175 101
1 : ['cherry-tree-leaves-and-fruits.jpg', '350', '300', 'Cherry leaf', '3', '114', '148', '235'] cropped_train//Cherry leaf/train_1.jpg
Cropped imgage shape (121, 145, 3) : 114 235 3 148  deltas: 121 145
2 : ['cherry-tree-leaves-and-fruits.jpg', '350', '300', 'Cherry leaf', '30', '184', '189', '297'] cropped_train//Cherry leaf/train_2.jpg
Cropped imgage shape (113, 159, 3) : 184 297 30 189  deltas: 113 159
3 : ['cherry-tree-leaves-and-fruits.jpg', '350', '300', 'Cherry leaf', '226', '4', '346', '83'] cropped_train//Cherry leaf/train_3.jpg
Cropped imgage shape (79, 120, 3) : 4 83 226 346  deltas: 79 120
4 : ['peach-and-leaf-stock-image-2809275.jpg', '1300', '1099', 'Peach leaf', '237', '479', '527', '810'] cropped_train//Peach leaf/train_4.jpg
Origi

Original imgage shape (2592, 3888, 3)
Cropped imgage shape (1000, 1048, 3) : 498 1498 1164 2212  deltas: 1000 1048
52 : ['img_0034.jpg', '3888', '2592', 'Strawberry leaf', '1160', '1302', '2088', '2342'] cropped_train//Strawberry leaf/train_52.jpg
Cropped imgage shape (1040, 928, 3) : 1302 2342 1160 2088  deltas: 1040 928
53 : ['img_0034.jpg', '3888', '2592', 'Strawberry leaf', '1848', '1214', '2808', '2134'] cropped_train//Strawberry leaf/train_53.jpg
Cropped imgage shape (920, 960, 3) : 1214 2134 1848 2808  deltas: 920 960
54 : ['img_0034.jpg', '3888', '2592', 'Strawberry leaf', '324', '510', '1344', '1514'] cropped_train//Strawberry leaf/train_54.jpg
Cropped imgage shape (1004, 1020, 3) : 510 1514 324 1344  deltas: 1004 1020
55 : ['img_0034.jpg', '3888', '2592', 'Strawberry leaf', '10', '1', '1540', '446'] cropped_train//Strawberry leaf/train_55.jpg
Cropped imgage shape (445, 1530, 3) : 1 446 10 1540  deltas: 445 1530
56 : ['stock-photo-apple-leaves-collection-isolated-on-white-back

Original imgage shape (3008, 2000, 3)
Cropped imgage shape (1069, 1262, 3) : 610 1679 738 2354  deltas: 1069 1616
105 : ['2f73110f80014a25a53f9551c94bf164.png.jpg', '0', '0', 'Corn Gray leaf spot', '58', '92', '525', '386'] cropped_train//Corn Gray leaf spot/train_105.jpg
Original imgage shape (395, 525, 3)
Cropped imgage shape (294, 467, 3) : 92 386 58 525  deltas: 294 467
106 : ['2373320.jpg', '450', '325', 'Tomato Early blight leaf', '134', '104', '402', '209'] cropped_train//Tomato Early blight leaf/train_106.jpg
Original imgage shape (325, 450, 3)
Cropped imgage shape (105, 268, 3) : 104 209 134 402  deltas: 105 268
107 : ['2373320.jpg', '450', '325', 'Tomato Early blight leaf', '104', '1', '193', '71'] cropped_train//Tomato Early blight leaf/train_107.jpg
Cropped imgage shape (70, 89, 3) : 1 71 104 193  deltas: 70 89
108 : ['2373320.jpg', '450', '325', 'Tomato Early blight leaf', '207', '214', '335', '302'] cropped_train//Tomato Early blight leaf/train_108.jpg
Cropped imgage shap

148 : ['corn-disease-update-fig-1-northern-leaf-blight.jpg', '1500', '2000', 'Corn leaf blight', '15', '240', '423', '2000'] cropped_train//Corn leaf blight/train_148.jpg
Cropped imgage shape (1760, 408, 3) : 240 2000 15 423  deltas: 1760 408
149 : ['Northern-Corn-Leaf-Blight-in-Sweet-Corn.jpg', '640', '640', 'Corn leaf blight', '114', '152', '639', '397'] cropped_train//Corn leaf blight/train_149.jpg
Original imgage shape (640, 640, 3)
Cropped imgage shape (245, 525, 3) : 152 397 114 639  deltas: 245 525
150 : ['p1070616.jpg', '2560', '1920', 'Squash Powdery mildew leaf', '682', '409', '1839', '1547'] cropped_train//Squash Powdery mildew leaf/train_150.jpg
Original imgage shape (1920, 2560, 3)
Cropped imgage shape (1138, 1157, 3) : 409 1547 682 1839  deltas: 1138 1157
151 : ['p1070616.jpg', '2560', '1920', 'Squash Powdery mildew leaf', '1', '1', '1412', '623'] cropped_train//Squash Powdery mildew leaf/train_151.jpg
Cropped imgage shape (622, 1411, 3) : 1 623 1 1412  deltas: 622 1411
1

Original imgage shape (1660, 2500, 3)
Cropped imgage shape (1364, 2480, 3) : 270 1634 20 2500  deltas: 1364 2480
210 : ['early_blight_spots_l.jpg', '400', '300', 'Tomato Early blight leaf', '1', '78', '400', '276'] cropped_train//Tomato Early blight leaf/train_210.jpg
Original imgage shape (300, 400, 3)
Cropped imgage shape (198, 399, 3) : 78 276 1 400  deltas: 198 399
211 : ['potato_late-blight_08_zoom.jpg', '900', '639', 'Potato leaf late blight', '38', '16', '474', '632'] cropped_train//Potato leaf late blight/train_211.jpg
Original imgage shape (639, 900, 3)
Cropped imgage shape (616, 436, 3) : 16 632 38 474  deltas: 616 436
212 : ['potato_late-blight_08_zoom.jpg', '900', '639', 'Potato leaf late blight', '493', '1', '889', '604'] cropped_train//Potato leaf late blight/train_212.jpg
Cropped imgage shape (603, 396, 3) : 1 604 493 889  deltas: 603 396
213 : ['blight-on-potato-leaves.jpg', '1000', '664', 'Potato leaf late blight', '524', '151', '788', '539'] cropped_train//Potato leaf

...

Original imgage shape (400, 400, 3)
Cropped imgage shape (285, 140, 3) : 62 347 260 400  deltas: 285 140
8197 : ['Blog-3.jpg', '400', '400', 'Potato leaf late blight', '164', '195', '266', '400'] cropped_train//Potato leaf late blight/train_8197.jpg
Cropped imgage shape (205, 102, 3) : 195 400 164 266  deltas: 205 102
8198 : ['Blog-3.jpg', '400', '400', 'Potato leaf late blight', '1', '46', '173', '150'] cropped_train//Potato leaf late blight/train_8198.jpg
Cropped imgage shape (104, 172, 3) : 46 150 1 173  deltas: 104 172
8199 : ['Blog-3.jpg', '400', '400', 'Potato leaf late blight', '139', '1', '304', '56'] cropped_train//Potato leaf late blight/train_8199.jpg
Cropped imgage shape (55, 165, 3) : 1 56 139 304  deltas: 55 165
8200 : ['0f3sA.jpg', '1032', '581', 'Bell_pepper leaf', '221', '228', '532', '388'] cropped_train//Bell_pepper leaf/train_8200.jpg
Original imgage shape (581, 1032, 3)
Cropped imgage shape (160, 311, 3) : 228 388 221 532  deltas: 160 311
8201 : ['0f3sA.jpg', '1032

Original imgage shape (2988, 5312, 3)
Cropped imgage shape (1443, 1077, 3) : 1347 2790 1278 2355  deltas: 1443 1077
8265 : ['20170524_195105-1k1eu8b.jpg', '5312', '2988', 'Apple rust leaf', '2836', '432', '4344', '1343'] cropped_train//Apple rust leaf/train_8265.jpg
Cropped imgage shape (911, 1508, 3) : 432 1343 2836 4344  deltas: 911 1508
8266 : ['20170524_195105-1k1eu8b.jpg', '5312', '2988', 'Apple rust leaf', '3444', '1305', '4432', '1905'] cropped_train//Apple rust leaf/train_8266.jpg
Cropped imgage shape (600, 988, 3) : 1305 1905 3444 4432  deltas: 600 988
8267 : ['20170524_195105-1k1eu8b.jpg', '5312', '2988', 'Apple rust leaf', '2290', '1416', '2986', '2459'] cropped_train//Apple rust leaf/train_8267.jpg
Cropped imgage shape (1043, 696, 3) : 1416 2459 2290 2986  deltas: 1043 696
8268 : ['20170524_195105-1k1eu8b.jpg', '5312', '2988', 'Apple rust leaf', '4009', '9', '4748', '609'] cropped_train//Apple rust leaf/train_8268.jpg
Cropped imgage shape (600, 739, 3) : 9 609 4009 4748  de

8336 : ['leaf_mold_tomato5x1200.jpg', '1190', '1989', 'Tomato mold leaf', '1', '59', '606', '732'] cropped_train//Tomato mold leaf/train_8336.jpg
Cropped imgage shape (673, 605, 3) : 59 732 1 606  deltas: 673 605
8337 : ['2017-08-27%2B-%2BLate%2BBlight%2Bon%2BGypsy%2B1.jpg', '1200', '1600', 'Tomato leaf late blight', '302', '163', '770', '1363'] cropped_train//Tomato leaf late blight/train_8337.jpg
Original imgage shape (1600, 1200, 3)
Cropped imgage shape (1200, 468, 3) : 163 1363 302 770  deltas: 1200 468
8338 : ['raspberryleafrust1.jpg', '1050', '797', 'Raspberry leaf', '1', '61', '874', '765'] cropped_train//Raspberry leaf/train_8338.jpg
Original imgage shape (797, 1050, 3)
Cropped imgage shape (704, 873, 3) : 61 765 1 874  deltas: 704 873
8339 : ['GIVXB.jpg', '800', '600', 'Tomato mold leaf', '61', '420', '229', '562'] cropped_train//Tomato mold leaf/train_8339.jpg
Original imgage shape (600, 800, 3)
Cropped imgage shape (142, 168, 3) : 420 562 61 229  deltas: 142 168
8340 : ['GIV

Original imgage shape (1024, 768, 3)
Cropped imgage shape (172, 270, 3) : 334 506 75 345  deltas: 172 270
8399 : ['v9hX3.jpg', '768', '1024', 'Bell_pepper leaf', '341', '388', '552', '626'] cropped_train//Bell_pepper leaf/train_8399.jpg
Cropped imgage shape (238, 211, 3) : 388 626 341 552  deltas: 238 211
8400 : ['v9hX3.jpg', '768', '1024', 'Bell_pepper leaf', '396', '539', '645', '764'] cropped_train//Bell_pepper leaf/train_8400.jpg
Cropped imgage shape (225, 249, 3) : 539 764 396 645  deltas: 225 249
8401 : ['v9hX3.jpg', '768', '1024', 'Bell_pepper leaf', '444', '3', '713', '242'] cropped_train//Bell_pepper leaf/train_8401.jpg
Cropped imgage shape (239, 269, 3) : 3 242 444 713  deltas: 239 269
8402 : ['v9hX3.jpg', '768', '1024', 'Bell_pepper leaf', '93', '171', '406', '305'] cropped_train//Bell_pepper leaf/train_8402.jpg
Cropped imgage shape (134, 313, 3) : 171 305 93 406  deltas: 134 313
8403 : ['bigseptoria01.jpg', '442', '298', 'Tomato Septoria leaf spot', '45', '62', '427', '244'

Original imgage shape (2592, 1936, 3)
Cropped imgage shape (604, 455, 3) : 1382 1986 842 1297  deltas: 604 455
8447 : ['img_20110302_170719.jpg', '1936', '2592', 'Strawberry leaf', '1001', '1164', '1557', '1527'] cropped_train//Strawberry leaf/train_8447.jpg
Cropped imgage shape (363, 556, 3) : 1164 1527 1001 1557  deltas: 363 556
8448 : ['img_20110302_170719.jpg', '1936', '2592', 'Strawberry leaf', '316', '1338', '934', '1790'] cropped_train//Strawberry leaf/train_8448.jpg
Cropped imgage shape (452, 618, 3) : 1338 1790 316 934  deltas: 452 618
8449 : ['stock-photo-green-soy-leaves-in-the-field-agricultural-background-31501516.jpg', '1500', '1225', 'Soyabean leaf', '829', '576', '1457', '902'] cropped_train//Soyabean leaf/train_8449.jpg
Original imgage shape (1225, 1500, 3)
Cropped imgage shape (326, 628, 3) : 576 902 829 1457  deltas: 326 628
8450 : ['stock-photo-green-soy-leaves-in-the-field-agricultural-background-31501516.jpg', '1500', '1225', 'Soyabean leaf', '298', '310', '712', 

In [ ]:
if not os.path.isdir(test_dir):
  CropImages(orig_data_dir=up_test_dir, dest_data_dir=test_dir, csv_file='up/test_labels.csv', include_original=False, prefix='test_')

0 : ['Black%20rot%20on%20foliage.jpg', '1060', '795', 'grape leaf black rot', '1', '58', '859', '732'] cropped_test//grape leaf black rot/test_0.jpg
Original imgage shape (795, 1060, 3)
Cropped imgage shape (674, 858, 3) : 58 732 1 859  deltas: 674 858
1 : ['pepper_leaf.jpg', '300', '400', 'Bell_pepper leaf', '40', '58', '251', '392'] cropped_test//Bell_pepper leaf/test_1.jpg
Original imgage shape (400, 300, 3)
Cropped imgage shape (334, 211, 3) : 58 392 40 251  deltas: 334 211
2 : ['depositphotos_1323264-Raspberry-leaf-on-white.jpg', '1024', '772', 'Raspberry leaf', '98', '75', '964', '710'] cropped_test//Raspberry leaf/test_2.jpg
Original imgage shape (772, 1024, 3)
Cropped imgage shape (635, 866, 3) : 75 710 98 964  deltas: 635 866
3 : ['dscn6689.jpg', '1024', '768', 'Tomato mold leaf', '140', '102', '887', '712'] cropped_test//Tomato mold leaf/test_3.jpg
Original imgage shape (768, 1024, 3)
Cropped imgage shape (610, 747, 3) : 102 712 140 887  deltas: 610 747
4 : ['IMG_1246.jpg', '

Cropped imgage shape (337, 1093, 3) : 478 815 72 1165  deltas: 337 1093
56 : ['28-500x375.jpg', '500', '375', 'Apple Scab Leaf', '149', '37', '460', '323'] cropped_test//Apple Scab Leaf/test_56.jpg
Original imgage shape (375, 500, 3)
Cropped imgage shape (286, 311, 3) : 37 323 149 460  deltas: 286 311
57 : ['28-500x375.jpg', '500', '375', 'Apple Scab Leaf', '51', '167', '191', '316'] cropped_test//Apple Scab Leaf/test_57.jpg
Cropped imgage shape (149, 140, 3) : 167 316 51 191  deltas: 149 140
58 : ['00.jpg', '480', '360', 'Bell_pepper leaf spot', '88', '26', '419', '259'] cropped_test//Bell_pepper leaf spot/test_58.jpg
Original imgage shape (360, 480, 3)
Cropped imgage shape (233, 331, 3) : 26 259 88 419  deltas: 233 331
59 : ['00.jpg', '480', '360', 'Bell_pepper leaf spot', '126', '251', '459', '357'] cropped_test//Bell_pepper leaf spot/test_59.jpg
Cropped imgage shape (106, 333, 3) : 251 357 126 459  deltas: 106 333
60 : ['00.jpg', '480', '360', 'Bell_pepper leaf spot', '1', '70', '1

104 : ['fungus-univ-of-minnesoeta.jpg', '800', '600', 'Tomato mold leaf', '93', '6', '425', '161'] cropped_test//Tomato mold leaf/test_104.jpg
Original imgage shape (600, 800, 3)
Cropped imgage shape (155, 332, 3) : 6 161 93 425  deltas: 155 332
105 : ['fungus-univ-of-minnesoeta.jpg', '800', '600', 'Tomato mold leaf', '122', '369', '377', '508'] cropped_test//Tomato mold leaf/test_105.jpg
Cropped imgage shape (139, 255, 3) : 369 508 122 377  deltas: 139 255
106 : ['fungus-univ-of-minnesoeta.jpg', '800', '600', 'Tomato mold leaf', '482', '208', '774', '504'] cropped_test//Tomato mold leaf/test_106.jpg
Cropped imgage shape (296, 292, 3) : 208 504 482 774  deltas: 296 292
107 : ['fungus-univ-of-minnesoeta.jpg', '800', '600', 'Tomato mold leaf', '129', '160', '649', '422'] cropped_test//Tomato mold leaf/test_107.jpg
Cropped imgage shape (262, 520, 3) : 160 422 129 649  deltas: 262 520
108 : ['0c.jpg', '1170', '501', 'Corn leaf blight', '6', '4', '1143', '495'] cropped_test//Corn leaf bligh

147 : ['strawberry-leaves-isolated-white-17473157.jpg', '1274', '1300', 'Strawberry leaf', '631', '696', '1091', '1145'] cropped_test//Strawberry leaf/test_147.jpg
Cropped imgage shape (449, 460, 3) : 696 1145 631 1091  deltas: 449 460
148 : ['strawberry-leaves-isolated-white-17473157.jpg', '1274', '1300', 'Strawberry leaf', '67', '767', '646', '1240'] cropped_test//Strawberry leaf/test_148.jpg
Cropped imgage shape (473, 579, 3) : 767 1240 67 646  deltas: 473 579
149 : ['strawberry-leaves-isolated-white-17473157.jpg', '1274', '1300', 'Strawberry leaf', '500', '120', '915', '513'] cropped_test//Strawberry leaf/test_149.jpg
Cropped imgage shape (393, 415, 3) : 120 513 500 915  deltas: 393 415
150 : ['strawberry-leaves-isolated-white-17473157.jpg', '1274', '1300', 'Strawberry leaf', '211', '196', '555', '515'] cropped_test//Strawberry leaf/test_150.jpg
Cropped imgage shape (319, 344, 3) : 196 515 211 555  deltas: 319 344
151 : ['strawberry-leaves-isolated-white-17473157.jpg', '1274', '130

Original imgage shape (4000, 6000, 3)
Cropped imgage shape (657, 1134, 3) : 1243 1900 3072 4206  deltas: 657 1134
188 : ['RARZ4.jpg', '6000', '4000', 'Tomato Septoria leaf spot', '3663', '1637', '5779', '2529'] cropped_test//Tomato Septoria leaf spot/test_188.jpg
Cropped imgage shape (892, 2116, 3) : 1637 2529 3663 5779  deltas: 892 2116
189 : ['RARZ4.jpg', '6000', '4000', 'Tomato Septoria leaf spot', '74', '995', '2548', '2551'] cropped_test//Tomato Septoria leaf spot/test_189.jpg
Cropped imgage shape (1556, 2474, 3) : 995 2551 74 2548  deltas: 1556 2474
190 : ['6447731_orig.jpg', '500', '332', 'Tomato Septoria leaf spot', '8', '4', '467', '298'] cropped_test//Tomato Septoria leaf spot/test_190.jpg
Original imgage shape (332, 500, 3)
Cropped imgage shape (294, 459, 3) : 4 298 8 467  deltas: 294 459
191 : ['Tomato-Mosaic-1wdr2jx-300x225.jpg', '130', '69', 'Tomato leaf mosaic virus', '2', '2', '129', '68'] cropped_test//Tomato leaf mosaic virus/test_191.jpg
Original imgage shape (69, 13

222 : ['three-vibrant-leaves-bird-cherry-tree-13905898.jpg', '1300', '972', 'Cherry leaf', '61', '126', '618', '712'] cropped_test//Cherry leaf/test_222.jpg
Original imgage shape (972, 1300, 3)
Cropped imgage shape (586, 557, 3) : 126 712 61 618  deltas: 586 557
223 : ['three-vibrant-leaves-bird-cherry-tree-13905898.jpg', '1300', '972', 'Cherry leaf', '550', '63', '838', '604'] cropped_test//Cherry leaf/test_223.jpg
Cropped imgage shape (541, 288, 3) : 63 604 550 838  deltas: 541 288
224 : ['three-vibrant-leaves-bird-cherry-tree-13905898.jpg', '1300', '972', 'Cherry leaf', '798', '136', '1175', '734'] cropped_test//Cherry leaf/test_224.jpg
Cropped imgage shape (598, 377, 3) : 136 734 798 1175  deltas: 598 377
225 : ['potato-blight-phytophthora-infestans-close-up-of-upper-surface-of-BMMRXC.jpg', '1300', '956', 'Potato leaf early blight', '59', '148', '1152', '788'] cropped_test//Potato leaf early blight/test_225.jpg
Original imgage shape (956, 1300, 3)
Cropped imgage shape (640, 1093, 3

Original imgage shape (5312, 2988, 3)
Cropped imgage shape (2366, 2272, 3) : 1289 3655 32 2304  deltas: 2366 2272
280 : ['20180511_090912-14gtw8a-e1526047952754.jpg', '2988', '5312', 'Apple leaf', '349', '361', '1704', '1128'] cropped_test//Apple leaf/test_280.jpg
Cropped imgage shape (767, 1355, 3) : 361 1128 349 1704  deltas: 767 1355
281 : ['5-29black-rot-chardRR.jpg', '450', '302', 'grape leaf black rot', '16', '3', '429', '302'] cropped_test//grape leaf black rot/test_281.jpg
Original imgage shape (302, 450, 3)
Cropped imgage shape (299, 413, 3) : 3 302 16 429  deltas: 299 413
282 : ['apple-leaf-closeup-37636177.jpg', '1300', '957', 'Apple leaf', '131', '52', '1047', '852'] cropped_test//Apple leaf/test_282.jpg
Original imgage shape (957, 1300, 3)
Cropped imgage shape (800, 916, 3) : 52 852 131 1047  deltas: 800 916
283 : ['6134794031202304.jpeg.jpg', '600', '800', 'Tomato Septoria leaf spot', '61', '203', '370', '668'] cropped_test//Tomato Septoria leaf spot/test_283.jpg
Original

Original imgage shape (2448, 3264, 3)
Cropped imgage shape (2187, 3182, 3) : 94 2281 82 3264  deltas: 2187 3182
339 : ['Corn-SCLB-2017-1.jpg', '3456', '5184', 'Corn leaf blight', '914', '22', '2764', '5150'] cropped_test//Corn leaf blight/test_339.jpg
Original imgage shape (5184, 3456, 3)
Cropped imgage shape (5128, 1850, 3) : 22 5150 914 2764  deltas: 5128 1850
340 : ['BIGSD.jpg', '450', '253', 'Tomato mold leaf', '396', '63', '436', '119'] cropped_test//Tomato mold leaf/test_340.jpg
Original imgage shape (253, 450, 3)
Cropped imgage shape (56, 40, 3) : 63 119 396 436  deltas: 56 40
341 : ['BIGSD.jpg', '450', '253', 'Tomato mold leaf', '350', '28', '393', '56'] cropped_test//Tomato mold leaf/test_341.jpg
Cropped imgage shape (28, 43, 3) : 28 56 350 393  deltas: 28 43
342 : ['BIGSD.jpg', '450', '253', 'Tomato mold leaf', '300', '36', '343', '64'] cropped_test//Tomato mold leaf/test_342.jpg
Cropped imgage shape (28, 43, 3) : 36 64 300 343  deltas: 28 43
343 : ['BIGSD.jpg', '450', '253',

377 : ['2256-body-1501555581-1.jpg', '3264', '2448', 'Corn rust leaf', '3', '8', '3239', '2416'] cropped_test//Corn rust leaf/test_377.jpg
Original imgage shape (2448, 3264, 3)
Cropped imgage shape (2408, 3236, 3) : 8 2416 3 3239  deltas: 2408 3236
378 : ['prunus-virginiana-le-gmittelhauser.jpg', '751', '1000', 'Cherry leaf', '36', '46', '724', '993'] cropped_test//Cherry leaf/test_378.jpg
Original imgage shape (1000, 751, 3)
Cropped imgage shape (947, 688, 3) : 46 993 36 724  deltas: 947 688
379 : ['grape-leaf-14309623.jpg', '1369', '1300', 'grape leaf', '135', '117', '1176', '1184'] cropped_test//grape leaf/test_379.jpg
Original imgage shape (1300, 1369, 3)
Cropped imgage shape (1067, 1041, 3) : 117 1184 135 1176  deltas: 1067 1041
380 : ['strawberry-leaves-stock-picture-948708.jpg', '1300', '1035', 'Strawberry leaf', '228', '45', '834', '458'] cropped_test//Strawberry leaf/test_380.jpg
Original imgage shape (1035, 1300, 3)
Cropped imgage shape (413, 606, 3) : 45 458 228 834  deltas:

430 : ['rose_tree_220430.jpg', '500', '500', 'Peach leaf', '126', '5', '348', '500'] cropped_test//Peach leaf/test_430.jpg
Original imgage shape (500, 500, 3)
Cropped imgage shape (495, 222, 3) : 5 500 126 348  deltas: 495 222
431 : ['7-17-Photo3_Septoria-MARY.jpg', '450', '359', 'Tomato Septoria leaf spot', '400', '229', '450', '348'] cropped_test//Tomato Septoria leaf spot/test_431.jpg
Original imgage shape (359, 450, 3)
Cropped imgage shape (119, 50, 3) : 229 348 400 450  deltas: 119 50
432 : ['7-17-Photo3_Septoria-MARY.jpg', '450', '359', 'Tomato Septoria leaf spot', '1', '3', '92', '281'] cropped_test//Tomato Septoria leaf spot/test_432.jpg
Cropped imgage shape (278, 91, 3) : 3 281 1 92  deltas: 278 91
433 : ['7-17-Photo3_Septoria-MARY.jpg', '450', '359', 'Tomato Septoria leaf spot', '119', '157', '179', '312'] cropped_test//Tomato Septoria leaf spot/test_433.jpg
Cropped imgage shape (155, 60, 3) : 157 312 119 179  deltas: 155 60
434 : ['7-17-Photo3_Septoria-MARY.jpg', '450', '359

In [ ]:
print(os.listdir(data_dir))
print(os.listdir(test_dir))
if not os.path.isdir('cropped_test/Potato leaf'):
  !mkdir 'cropped_test/Potato leaf'
if not os.path.isdir('cropped_test/Tomato two spotted spider mites leaf'):
  !mkdir 'cropped_test/Tomato two spotted spider mites leaf'

['Cherry leaf', 'Peach leaf', 'Apple rust leaf', 'Potato leaf late blight', 'Strawberry leaf', 'Corn rust leaf', 'Tomato leaf late blight', 'Tomato mold leaf', 'Potato leaf early blight', 'Apple leaf', 'Tomato leaf yellow virus', 'Blueberry leaf', 'Tomato leaf mosaic virus', 'Raspberry leaf', 'Corn leaf blight', 'Tomato leaf bacterial spot', 'Squash Powdery mildew leaf', 'grape leaf', 'Corn Gray leaf spot', 'Tomato Early blight leaf', 'Apple Scab Leaf', 'Tomato Septoria leaf spot', 'Tomato leaf', 'Soyabean leaf', 'Bell_pepper leaf spot', 'Bell_pepper leaf', 'grape leaf black rot', 'Potato leaf', 'Tomato two spotted spider mites leaf']
['grape leaf black rot', 'Bell_pepper leaf', 'Raspberry leaf', 'Tomato mold leaf', 'Tomato leaf', 'Corn rust leaf', 'Bell_pepper leaf spot', 'grape leaf', 'Cherry leaf', 'Apple leaf', 'Squash Powdery mildew leaf', 'Peach leaf', 'Potato leaf early blight', 'Tomato leaf bacterial spot', 'Potato leaf late blight', 'Blueberry leaf', 'Corn leaf blight', 'Tomat

In [ ]:
if not os.path.isdir(all_dir):
  !mkdir cropped_all
  ! cp -r cropped_test/* cropped_all/
  ! cp -r cropped_train/* cropped_all/

In [ ]:
datagen = cai.util.create_image_generator(rotation_range=20, width_shift_range=0.3, height_shift_range=0.3, channel_shift_range=0.1) 

def lrscheduler(epoch):
  return 0.01 * (0.97**epoch)

In [ ]:
num_classes = 29
batch_size = 32
epochs = 75
target_size_x = 224 # default value is 224
target_size_y = 224 # default value is 224

In [ ]:
train_x, val_x, test_x, train_y, val_y, test_y, classweight, classes = cai.datasets.load_images_from_folders(seed=7,
  root_dir=all_dir, lab=False, 
  verbose=True, bipolar=False, base_model_name='plant_doc',
  has_training=True, has_validation=True, has_testing=True,
  smart_resize=True,
  training_size=0.65, validation_size=0.15, test_size=0.2,
  target_size=(target_size_x, target_size_y))

print(train_x.shape,val_x.shape,test_x.shape)
print(train_y.shape,val_y.shape,test_y.shape)

Loading  29  classes.
smart resize is enabled.
loading train images
train shape is: (5761, 224, 224, 3)
loading validation images
validation shape is: (1333, 224, 224, 3)
loading test images
test shape is: (1789, 224, 224, 3)
Channel  0  min: 0.0  max: 1.0
Channel  1  min: 0.0  max: 1.0
Channel  2  min: 0.0  max: 1.0
Loaded.
(5761, 224, 224, 3) (1333, 224, 224, 3) (1789, 224, 224, 3)
(5761, 29) (1333, 29) (1789, 29)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass classes=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28], y=[ 0  0  0 ... 28 28 28] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [ ]:
def work_on_efficientnet(show_model=False, run_fit=False, test_results=False, calc_f1=False):
  monitor='val_accuracy'
  if (calc_f1): 
    test_results=True
  if (show_model):
    input_shape = (target_size_x, target_size_y, 3)
  else:
    input_shape = (None, None, 3)
  deep_two_paths_compression=0.6
  for kType in [-1, 1, 2]:
      basefilename = 'DEV-EfficientNet-CroppedPlantDoc-'+str(kType)
      best_result_file_name = basefilename+'-best_result.hdf5'
      print('Running: '+basefilename)
      
      if kType==-1:
        model = cai.efficientnet.EfficientNetB0(
          include_top=True,
          input_shape=input_shape,
          classes=num_classes)
      else:
        model = cai.efficientnet.kEfficientNetB0(
          include_top=True,
          input_shape=input_shape,
          classes=num_classes,
          kType=kType)
      
      optimizer = keras.optimizers.RMSprop()
      optimizer = mixed_precision.LossScaleOptimizer(optimizer)

      model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'])

      if (show_model): model.summary()

      save_best = keras.callbacks.ModelCheckpoint(
            filepath=best_result_file_name,
            monitor=monitor,
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            save_freq='epoch')

      if (run_fit):
        history = model.fit(
          x = datagen.flow(train_x, train_y, batch_size=batch_size),
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(val_x, val_y),
          callbacks=[save_best, tf.keras.callbacks.LearningRateScheduler(lrscheduler)],
          workers=max([multiprocessing.cpu_count(), 4])
          )
      if (test_results):
        print('Best Model Results: '+basefilename)
        model = cai.models.load_kereas_model(best_result_file_name)
        evaluated = model.evaluate(test_x,test_y)
        for metric, name in zip(evaluated,["loss","acc"]):
              print(name,metric)
      if (calc_f1):
        pred_y = model.predict(test_x)
        print("Predicted Shape:", pred_y.shape)
        pred_classes_y = np.array(list(np.argmax(pred_y, axis=1)))
        test_classes_y = np.array(list(np.argmax(test_y, axis=1)))
        print("Pred classes shape:",pred_classes_y.shape)
        print("Test classes shape:",test_classes_y.shape)
        report = classification_report(test_classes_y, pred_classes_y, digits=4)
        print(report)
      print('Finished: '+basefilename)

# Show Models

In [ ]:
work_on_efficientnet(show_model=True, run_fit=False, test_results=False)

Running: DEV-EfficientNet-CroppedPlantDoc--1
Model: "efficientnet-b0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         stem_conv_pad[0][0]              
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
_______________________________________

Model: "efficientnet-b0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         stem_conv_pad[0][0]              
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
____________________________________________________________________________________

Model: "efficientnet-b0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         stem_conv_pad[0][0]              
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
____________________________________________________________________________________

# Fitting

In [ ]:
work_on_efficientnet(show_model=False, run_fit=True, test_results=True)

Running: DEV-EfficientNet-CroppedPlantDoc--1
Epoch 1/75
181/181 [==============================] - 45s 161ms/step - loss: 4.3951 - accuracy: 0.0792 - val_loss: nan - val_accuracy: 0.0473

Epoch 00001: val_accuracy improved from -inf to 0.04726, saving model to DEV-EfficientNet-CroppedPlantDoc--1-best_result.hdf5
Epoch 2/75
181/181 [==============================] - 27s 146ms/step - loss: 3.2685 - accuracy: 0.0777 - val_loss: nan - val_accuracy: 0.0383

Epoch 00002: val_accuracy did not improve from 0.04726
Epoch 3/75
181/181 [==============================] - 27s 147ms/step - loss: 3.1846 - accuracy: 0.0842 - val_loss: 4.0662 - val_accuracy: 0.0885

Epoch 00003: val_accuracy improved from 0.04726 to 0.08852, saving model to DEV-EfficientNet-CroppedPlantDoc--1-best_result.hdf5
Epoch 4/75
181/181 [==============================] - 27s 147ms/step - loss: 3.0967 - accuracy: 0.1046 - val_loss: 10.6554 - val_accuracy: 0.0638

Epoch 00004: val_accuracy did not improve from 0.08852
Epoch 5/75



Epoch 00035: val_accuracy improved from 0.39235 to 0.39610, saving model to DEV-EfficientNet-CroppedPlantDoc--1-best_result.hdf5
Epoch 36/75
181/181 [==============================] - 27s 146ms/step - loss: 1.8945 - accuracy: 0.4118 - val_loss: 2.0835 - val_accuracy: 0.3766

Epoch 00036: val_accuracy did not improve from 0.39610
Epoch 37/75
181/181 [==============================] - 27s 146ms/step - loss: 1.8816 - accuracy: 0.4143 - val_loss: 1.9177 - val_accuracy: 0.4209

Epoch 00037: val_accuracy improved from 0.39610 to 0.42086, saving model to DEV-EfficientNet-CroppedPlantDoc--1-best_result.hdf5
Epoch 38/75
181/181 [==============================] - 27s 146ms/step - loss: 1.8566 - accuracy: 0.4166 - val_loss: 2.1561 - val_accuracy: 0.3473

Epoch 00038: val_accuracy did not improve from 0.42086
Epoch 39/75
181/181 [==============================] - 27s 146ms/step - loss: 1.8641 - accuracy: 0.4087 - val_loss: 1.9272 - val_accuracy: 0.4254

Epoch 00039: val_accuracy improved from 0.4


Epoch 00072: val_accuracy did not improve from 0.50188
Epoch 73/75
181/181 [==============================] - 27s 146ms/step - loss: 1.4717 - accuracy: 0.5289 - val_loss: 1.6393 - val_accuracy: 0.4989

Epoch 00073: val_accuracy did not improve from 0.50188
Epoch 74/75
181/181 [==============================] - 27s 146ms/step - loss: 1.4787 - accuracy: 0.5240 - val_loss: 1.6259 - val_accuracy: 0.4929

Epoch 00074: val_accuracy did not improve from 0.50188
Epoch 75/75
181/181 [==============================] - 27s 146ms/step - loss: 1.4426 - accuracy: 0.5476 - val_loss: 1.6213 - val_accuracy: 0.4951

Epoch 00075: val_accuracy did not improve from 0.50188
Best Model Results: DEV-EfficientNet-CroppedPlantDoc--1
56/56 [==============================] - 3s 36ms/step - loss: 1.6268 - accuracy: 0.4908
loss 1.6268210411071777
acc 0.4907769560813904
Finished: DEV-EfficientNet-CroppedPlantDoc--1
Running: DEV-EfficientNet-CroppedPlantDoc-1
Epoch 1/75
181/181 [==============================] - 128

181/181 [==============================] - 71s 392ms/step - loss: 1.4329 - accuracy: 0.5446 - val_loss: 1.7139 - val_accuracy: 0.4846

Epoch 00031: val_accuracy did not improve from 0.52363
Epoch 32/75
181/181 [==============================] - 72s 393ms/step - loss: 1.3747 - accuracy: 0.5711 - val_loss: 1.8015 - val_accuracy: 0.4839

Epoch 00032: val_accuracy did not improve from 0.52363
Epoch 33/75
181/181 [==============================] - 72s 394ms/step - loss: 1.3589 - accuracy: 0.5785 - val_loss: 1.6082 - val_accuracy: 0.5221

Epoch 00033: val_accuracy did not improve from 0.52363
Epoch 34/75
181/181 [==============================] - 71s 391ms/step - loss: 1.3156 - accuracy: 0.5755 - val_loss: 1.5261 - val_accuracy: 0.5431

Epoch 00034: val_accuracy improved from 0.52363 to 0.54314, saving model to DEV-EfficientNet-CroppedPlantDoc-1-best_result.hdf5
Epoch 35/75
181/181 [==============================] - 71s 391ms/step - loss: 1.2761 - accuracy: 0.5972 - val_loss: 1.7270 - val_ac

181/181 [==============================] - 71s 390ms/step - loss: 0.6505 - accuracy: 0.7826 - val_loss: 1.3517 - val_accuracy: 0.6542

Epoch 00068: val_accuracy improved from 0.63766 to 0.65416, saving model to DEV-EfficientNet-CroppedPlantDoc-1-best_result.hdf5
Epoch 69/75
181/181 [==============================] - 71s 391ms/step - loss: 0.6328 - accuracy: 0.7950 - val_loss: 1.4063 - val_accuracy: 0.6362

Epoch 00069: val_accuracy did not improve from 0.65416
Epoch 70/75
181/181 [==============================] - 72s 394ms/step - loss: 0.6661 - accuracy: 0.7808 - val_loss: 1.4193 - val_accuracy: 0.6347

Epoch 00070: val_accuracy did not improve from 0.65416
Epoch 71/75
181/181 [==============================] - 73s 398ms/step - loss: 0.6230 - accuracy: 0.7964 - val_loss: 1.4350 - val_accuracy: 0.6302

Epoch 00071: val_accuracy did not improve from 0.65416
Epoch 72/75
181/181 [==============================] - 73s 399ms/step - loss: 0.6150 - accuracy: 0.8000 - val_loss: 1.4397 - val_ac

181/181 [==============================] - 76s 414ms/step - loss: 1.6627 - accuracy: 0.4758 - val_loss: 1.6814 - val_accuracy: 0.4914

Epoch 00027: val_accuracy improved from 0.45161 to 0.49137, saving model to DEV-EfficientNet-CroppedPlantDoc-2-best_result.hdf5
Epoch 28/75
181/181 [==============================] - 76s 414ms/step - loss: 1.6486 - accuracy: 0.4783 - val_loss: 1.7949 - val_accuracy: 0.4614

Epoch 00028: val_accuracy did not improve from 0.49137
Epoch 29/75
181/181 [==============================] - 76s 415ms/step - loss: 1.6261 - accuracy: 0.4779 - val_loss: 1.7246 - val_accuracy: 0.4884

Epoch 00029: val_accuracy did not improve from 0.49137
Epoch 30/75
181/181 [==============================] - 76s 417ms/step - loss: 1.6000 - accuracy: 0.4900 - val_loss: 1.8489 - val_accuracy: 0.4719

Epoch 00030: val_accuracy did not improve from 0.49137
Epoch 31/75
181/181 [==============================] - 75s 414ms/step - loss: 1.5277 - accuracy: 0.5200 - val_loss: 1.5656 - val_ac

181/181 [==============================] - 76s 414ms/step - loss: 0.9044 - accuracy: 0.7113 - val_loss: 1.4298 - val_accuracy: 0.5979

Epoch 00064: val_accuracy did not improve from 0.61065
Epoch 65/75
181/181 [==============================] - 76s 417ms/step - loss: 0.8564 - accuracy: 0.7162 - val_loss: 1.4163 - val_accuracy: 0.6047

Epoch 00065: val_accuracy did not improve from 0.61065
Epoch 66/75
181/181 [==============================] - 76s 416ms/step - loss: 0.8491 - accuracy: 0.7214 - val_loss: 1.4125 - val_accuracy: 0.5979

Epoch 00066: val_accuracy did not improve from 0.61065
Epoch 67/75
181/181 [==============================] - 76s 415ms/step - loss: 0.8303 - accuracy: 0.7267 - val_loss: 1.3806 - val_accuracy: 0.6144

Epoch 00067: val_accuracy improved from 0.61065 to 0.61440, saving model to DEV-EfficientNet-CroppedPlantDoc-2-best_result.hdf5
Epoch 68/75
181/181 [==============================] - 76s 415ms/step - loss: 0.8349 - accuracy: 0.7260 - val_loss: 1.3685 - val_ac

# Test Results

In [ ]:
work_on_efficientnet(show_model=False, run_fit=False, test_results=True)

Running: DEV-EfficientNet-CroppedPlantDoc--1
Best Model Results: DEV-EfficientNet-CroppedPlantDoc--1
56/56 [==============================] - 3s 35ms/step - loss: 1.6268 - accuracy: 0.4908
loss 1.6268210411071777
acc 0.4907769560813904
Finished: DEV-EfficientNet-CroppedPlantDoc--1
Running: DEV-EfficientNet-CroppedPlantDoc-1
Best Model Results: DEV-EfficientNet-CroppedPlantDoc-1
56/56 [==============================] - 13s 68ms/step - loss: 1.2738 - accuracy: 0.6646
loss 1.2738487720489502
acc 0.664617121219635
Finished: DEV-EfficientNet-CroppedPlantDoc-1
Running: DEV-EfficientNet-CroppedPlantDoc-2
Best Model Results: DEV-EfficientNet-CroppedPlantDoc-2
56/56 [==============================] - 13s 72ms/step - loss: 1.2653 - accuracy: 0.6439
loss 1.265318751335144
acc 0.6439351439476013
Finished: DEV-EfficientNet-CroppedPlantDoc-2
